In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2    
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    sa1 = (torch.sum((model(xa1)-u_ex(xa1))**2)/xa1.size()[0]).cuda() 
    sa2 = (torch.sum((model(xa2)-u_ex(xa2))**2)/xa1.size()[0]).cuda()  
    sb1 = (torch.sum((model(xb1)-u_ex(xb1))**2)/xa1.size()[0]).cuda()  
    sb2 = (torch.sum((model(xb2)-u_ex(xb2))**2)/xa1.size()[0]).cuda() 
    sc1 = (torch.sum((model(xc1)-u_ex(xc1))**2)/xa1.size()[0]).cuda()  
    sc2 = (torch.sum((model(xc2)-u_ex(xc2))**2)/xa1.size()[0]).cuda() 
    sd1 = (torch.sum((model(xd1)-u_ex(xd1))**2)/xa1.size()[0]).cuda()  
    sd2 = (torch.sum((model(xd2)-u_ex(xd2))**2)/xa1.size()[0]).cuda() 
    part_2 = sa1 + sa2 + sb1 + sb2 + sc1 + sc2 + sd1 + sd2
    
    lambda1 = 0.1
    return part_1 + lambda1 * part_2 / 8

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_4D_Dirichlet_lambda0.1.npy", error_save)
np.save("DGM_relative_error_4D_Dirichlet_lambda0.1.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(831.2715, device='cuda:0')
current relative error is:  tensor(1.1581, device='cuda:0')
current epoch is:  50
current loss is:  tensor(461.8770, device='cuda:0')
current relative error is:  tensor(0.8294, device='cuda:0')
current epoch is:  100
current loss is:  tensor(73.2377, device='cuda:0')
current relative error is:  tensor(0.3405, device='cuda:0')
current epoch is:  150
current loss is:  tensor(20.7152, device='cuda:0')
current relative error is:  tensor(0.6191, device='cuda:0')
current epoch is:  200
current loss is:  tensor(15.8710, device='cuda:0')
current relative error is:  tensor(0.7147, device='cuda:0')
current epoch is:  250
current loss is:  tensor(12.8277, device='cuda:0')
current relative error is:  tensor(0.7257, device='cuda:0')
current epoch is:  300
current loss is:  tensor(12.7516, device='cuda:0')
current relative error is:  tensor(0.7218, device='cuda:0')
current epoch is:  350
current loss is:  tensor(11.4979, device

current epoch is:  3100
current loss is:  tensor(7.9520, device='cuda:0')
current relative error is:  tensor(0.6937, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(8.1930, device='cuda:0')
current relative error is:  tensor(0.6778, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(8.5480, device='cuda:0')
current relative error is:  tensor(0.6801, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(7.9421, device='cuda:0')
current relative error is:  tensor(0.6915, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(8.0653, device='cuda:0')
current relative error is:  tensor(0.6725, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(8.2332, device='cuda:0')
current relative error is:  tensor(0.6888, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(7.8822, device='cuda:0')
current relative error is:  tensor(0.6641, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(8.3143, devic

current epoch is:  6200
current loss is:  tensor(1.3968, device='cuda:0')
current relative error is:  tensor(0.4440, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(1.2372, device='cuda:0')
current relative error is:  tensor(0.4382, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(1.2009, device='cuda:0')
current relative error is:  tensor(0.4304, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(1.1453, device='cuda:0')
current relative error is:  tensor(0.4309, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(1.0609, device='cuda:0')
current relative error is:  tensor(0.4238, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(1.0079, device='cuda:0')
current relative error is:  tensor(0.4206, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(1.0384, device='cuda:0')
current relative error is:  tensor(0.4167, device='cuda:0')
current epoch is:  6550
current loss is:  tensor(1.0718, devic

current epoch is:  9300
current loss is:  tensor(0.2305, device='cuda:0')
current relative error is:  tensor(0.3556, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(0.2386, device='cuda:0')
current relative error is:  tensor(0.3621, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(0.2213, device='cuda:0')
current relative error is:  tensor(0.3525, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(0.2499, device='cuda:0')
current relative error is:  tensor(0.3526, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(0.2464, device='cuda:0')
current relative error is:  tensor(0.3510, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(0.2322, device='cuda:0')
current relative error is:  tensor(0.3634, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(0.2251, device='cuda:0')
current relative error is:  tensor(0.3553, device='cuda:0')
current epoch is:  9650
current loss is:  tensor(0.2755, devic

current epoch is:  12350
current loss is:  tensor(0.1623, device='cuda:0')
current relative error is:  tensor(0.3262, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(0.1578, device='cuda:0')
current relative error is:  tensor(0.3268, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(0.1444, device='cuda:0')
current relative error is:  tensor(0.3333, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(0.1710, device='cuda:0')
current relative error is:  tensor(0.3285, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(0.1785, device='cuda:0')
current relative error is:  tensor(0.3269, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(0.1707, device='cuda:0')
current relative error is:  tensor(0.3211, device='cuda:0')
current epoch is:  12650
current loss is:  tensor(0.1529, device='cuda:0')
current relative error is:  tensor(0.3295, device='cuda:0')
current epoch is:  12700
current loss is:  tensor(0.143

current epoch is:  15400
current loss is:  tensor(0.1699, device='cuda:0')
current relative error is:  tensor(0.3160, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(0.1153, device='cuda:0')
current relative error is:  tensor(0.3117, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(0.1400, device='cuda:0')
current relative error is:  tensor(0.3038, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(0.1128, device='cuda:0')
current relative error is:  tensor(0.3116, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(0.1179, device='cuda:0')
current relative error is:  tensor(0.3084, device='cuda:0')
current epoch is:  15650
current loss is:  tensor(0.1261, device='cuda:0')
current relative error is:  tensor(0.3048, device='cuda:0')
current epoch is:  15700
current loss is:  tensor(0.1302, device='cuda:0')
current relative error is:  tensor(0.3048, device='cuda:0')
current epoch is:  15750
current loss is:  tensor(0.123

current epoch is:  18450
current loss is:  tensor(0.0980, device='cuda:0')
current relative error is:  tensor(0.2906, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(0.1002, device='cuda:0')
current relative error is:  tensor(0.2950, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(0.0954, device='cuda:0')
current relative error is:  tensor(0.2891, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(0.1013, device='cuda:0')
current relative error is:  tensor(0.2927, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(0.0983, device='cuda:0')
current relative error is:  tensor(0.2885, device='cuda:0')
current epoch is:  18700
current loss is:  tensor(0.1256, device='cuda:0')
current relative error is:  tensor(0.2873, device='cuda:0')
current epoch is:  18750
current loss is:  tensor(0.0980, device='cuda:0')
current relative error is:  tensor(0.2848, device='cuda:0')
current epoch is:  18800
current loss is:  tensor(0.094

In [18]:
error_save_append = np.zeros(traintime)
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save_append[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_4D_Dirichlet_lambda0.1_append.npy", error_save_append)
np.save("DGM_relative_error_4D_Dirichlet_lambda0.1_append.npy", error_save_append)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(0.1595, device='cuda:0')
current relative error is:  tensor(0.2958, device='cuda:0')
current epoch is:  50
current loss is:  tensor(0.1025, device='cuda:0')
current relative error is:  tensor(0.2832, device='cuda:0')
current epoch is:  100
current loss is:  tensor(0.1355, device='cuda:0')
current relative error is:  tensor(0.2804, device='cuda:0')
current epoch is:  150
current loss is:  tensor(0.0927, device='cuda:0')
current relative error is:  tensor(0.2763, device='cuda:0')
current epoch is:  200
current loss is:  tensor(0.0898, device='cuda:0')
current relative error is:  tensor(0.2833, device='cuda:0')
current epoch is:  250
current loss is:  tensor(0.0889, device='cuda:0')
current relative error is:  tensor(0.2697, device='cuda:0')
current epoch is:  300
current loss is:  tensor(0.0882, device='cuda:0')
current relative error is:  tensor(0.2874, device='cuda:0')
current epoch is:  350
current loss is:  tensor(0.1129, device='cuda:0')

current epoch is:  3100
current loss is:  tensor(0.0747, device='cuda:0')
current relative error is:  tensor(0.2680, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(0.1045, device='cuda:0')
current relative error is:  tensor(0.2603, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(0.0897, device='cuda:0')
current relative error is:  tensor(0.2674, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(0.0975, device='cuda:0')
current relative error is:  tensor(0.2595, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(0.1132, device='cuda:0')
current relative error is:  tensor(0.2651, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(0.0811, device='cuda:0')
current relative error is:  tensor(0.2694, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(0.0822, device='cuda:0')
current relative error is:  tensor(0.2672, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(0.0732, devic

current epoch is:  6200
current loss is:  tensor(0.0683, device='cuda:0')
current relative error is:  tensor(0.2581, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(0.1442, device='cuda:0')
current relative error is:  tensor(0.2624, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(0.0839, device='cuda:0')
current relative error is:  tensor(0.2524, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(0.0712, device='cuda:0')
current relative error is:  tensor(0.2577, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(0.0880, device='cuda:0')
current relative error is:  tensor(0.2545, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(0.0705, device='cuda:0')
current relative error is:  tensor(0.2556, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(0.1464, device='cuda:0')
current relative error is:  tensor(0.2567, device='cuda:0')
current epoch is:  6550
current loss is:  tensor(0.1111, devic

current epoch is:  9300
current loss is:  tensor(0.0730, device='cuda:0')
current relative error is:  tensor(0.2518, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(0.0608, device='cuda:0')
current relative error is:  tensor(0.2440, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(0.0553, device='cuda:0')
current relative error is:  tensor(0.2403, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(0.0612, device='cuda:0')
current relative error is:  tensor(0.2460, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(0.0876, device='cuda:0')
current relative error is:  tensor(0.2478, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(0.0755, device='cuda:0')
current relative error is:  tensor(0.2466, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(0.0663, device='cuda:0')
current relative error is:  tensor(0.2469, device='cuda:0')
current epoch is:  9650
current loss is:  tensor(0.0894, devic

current epoch is:  12350
current loss is:  tensor(0.0603, device='cuda:0')
current relative error is:  tensor(0.2362, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(0.0698, device='cuda:0')
current relative error is:  tensor(0.2338, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(0.0564, device='cuda:0')
current relative error is:  tensor(0.2397, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(0.0556, device='cuda:0')
current relative error is:  tensor(0.2284, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(0.0524, device='cuda:0')
current relative error is:  tensor(0.2286, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(0.0677, device='cuda:0')
current relative error is:  tensor(0.2389, device='cuda:0')
current epoch is:  12650
current loss is:  tensor(0.0611, device='cuda:0')
current relative error is:  tensor(0.2341, device='cuda:0')
current epoch is:  12700
current loss is:  tensor(0.067

current epoch is:  15400
current loss is:  tensor(0.0563, device='cuda:0')
current relative error is:  tensor(0.2280, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(0.0510, device='cuda:0')
current relative error is:  tensor(0.2279, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(0.0516, device='cuda:0')
current relative error is:  tensor(0.2238, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(0.0558, device='cuda:0')
current relative error is:  tensor(0.2334, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(0.0481, device='cuda:0')
current relative error is:  tensor(0.2264, device='cuda:0')
current epoch is:  15650
current loss is:  tensor(0.0776, device='cuda:0')
current relative error is:  tensor(0.2321, device='cuda:0')
current epoch is:  15700
current loss is:  tensor(0.0474, device='cuda:0')
current relative error is:  tensor(0.2288, device='cuda:0')
current epoch is:  15750
current loss is:  tensor(0.060

current epoch is:  18450
current loss is:  tensor(0.0493, device='cuda:0')
current relative error is:  tensor(0.2262, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(0.0480, device='cuda:0')
current relative error is:  tensor(0.2268, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(0.0434, device='cuda:0')
current relative error is:  tensor(0.2258, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(0.0550, device='cuda:0')
current relative error is:  tensor(0.2281, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(0.0828, device='cuda:0')
current relative error is:  tensor(0.2180, device='cuda:0')
current epoch is:  18700
current loss is:  tensor(0.0541, device='cuda:0')
current relative error is:  tensor(0.2206, device='cuda:0')
current epoch is:  18750
current loss is:  tensor(0.0633, device='cuda:0')
current relative error is:  tensor(0.2229, device='cuda:0')
current epoch is:  18800
current loss is:  tensor(0.079